In [1]:
import sqlalchemy, pyodbc
import pandas as pd
import pymysql.cursors
from sqlalchemy import create_engine
import psycopg2
import numpy

#### Create the connection to servers from where we will access our data

In [2]:
conn_extract_Servers = pyodbc.connect(
    r'DRIVER={SQL Server Native Client 11.0};'
    r'SERVER=cds-hub-11.ad.ea.com;'
    r'DATABASE=BUGHUB;'
    r'Trusted_Connection=yes;'
    )

In [3]:
## 33 sec. Used when comparing src vs Bughub
print('Picking Bugs from Bughub')
existing = pd.read_sql("""select distinct BugIdUnique as id, BugId as issuenum, ProjectId as project, SourceServerId as source from staging.Bugs
where IsDeleted = 0""", con = conn_extract_Servers)

Picking Bugs from Bughub


In [4]:
print('Bugs from Bughub Imported Successfully')
print('Picking Source Servers List')
df = pd.read_sql("""SELECT * from nrtetl.etlsources s where isactive = 1 and SourceSystem in ('Jira','Devtrack') and ID <> 33""", con=conn_extract_Servers)

Bugs from Bughub Imported Successfully
Picking Source Servers List


#### Create a dictionary which contains the templates of the connection strings

In [5]:
templates = {
    'MSSQL_PASS': r'DRIVER={{SQL Server Native Client 11.0}};SERVER={SERVER};DATABASE={DATABASE};UID={UID};PWD={PWD};',
    'MSSQL_I': r'DRIVER={{SQL Server Native Client 11.0}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;',
    'MySQL': r'host={host};db={db};user={user};password={password};',
    'PostgreSql_PASS': r'host={host};dbname={dbname};user={user};password={password};'
}

#### Tables that will need to be accessed with the query

In [6]:
lookup ={'Jira':'jiraissue' ,'Devtrack':'bug'}

#### Create a dictionary which contains all the information about the servers that need to be accessed

In [7]:
connections = {}
for index, row in df.iterrows():
    if row['ServerType'] == 'SqlServer':
        ## SQL SERVER PASS
        if 'User' in str(row['ConnectionString']):
            values = {}
            for ix in row['ConnectionString'].split(';'):
                if 'Data Source' in ix:
                    values['SERVER']= ix.split('=')[1]
                if 'Initial Catalog' in ix:
                    values['DATABASE'] = ix.split('=')[1]
                if 'User' in ix:
                    values['UID'] = ix.split('=')[1]
                if 'Password' in ix:
                    try:
                        values['PWD'] = ix.split(':')[1]
                    except:
                        values['PWD'] = ix.split('=')[1]
            connections[row['Id']]=['MSSQL',lookup[row['SourceSystem']],templates['MSSQL_PASS'].format(**values)]
        else:
            ## SQL SERVER I
            values = {}
            for ix in row['ConnectionString'].split(';'):
                if 'Data Source' in ix:
                    values['SERVER']= ix.split('=')[1]#.split(',')[0]
                if 'Initial Catalog' in ix:
                    values['DATABASE'] = ix.split('=')[1]
            connections[row['Id']]=['MSSQL',lookup[row['SourceSystem']],templates['MSSQL_I'].format(**values)]
    if row['ServerType'] == 'MySql':
        ## MySql
        if 'user' in str(row['ConnectionString']):
            values = {}
            for ix in row['ConnectionString'].split(';'):
                if 'server' in ix:
                    values['host']= ix.split('=')[1]
                if 'database' in ix:
                    values['db'] = ix.split('=')[1]
                if 'user' in ix:
                    values['user'] = ix.split('=')[1]
                if 'password' in ix:
                    values['password'] = ix.split('d=')[1]
            connections[row['Id']]=['MySQL',lookup[row['SourceSystem']],values]
    if row['ServerType'] == 'PostgreSql':
        ## PostgreSql
        if 'Password' in str(row['ConnectionString']):
            values = {}
            for ix in row['ConnectionString'].split(';'):
                if 'Server' in ix:
                    values['host']= ix.split('=')[1]
                if 'Database' in ix:
                    values['dbname'] = ix.split('=')[1]
                if 'User' in ix:
                    values['user'] = ix.split('=')[1]
                if 'Password' in ix:
                    values['password'] = ix.split('=')[1]
            connections[row['Id']]=['PostgreSql',lookup[row['SourceSystem']],values]
        else:
            ## PostgreSql wthout PASS
            values = {}
            for ix in row['ConnectionString'].split(';'):
                if 'Server' in ix:
                    values['host']= ix.split('=')[1]
                if 'Database' in ix:
                    values['dbname'] = ix.split('=')[1]
                if 'User' in ix:
                    values['user'] = ix.split('=')[1]
            connections[row['Id']]=['PostgreSqlPass',lookup[row['SourceSystem']],values]

#### Create a function which loops through the servers and execute a query based on the server type (SQL,MySQL,PostgreSQL)

In [8]:
def query(server,servertype,connstring,table):
    if servertype == 'MSSQL':
        if table =='jiraissue':
            try:
                if server == 27:
                    df = pd.read_sql('select id ,issuenum , project from {} where ID not in (SELECT DISTINCT ID FROM {} WHERE Security IS not NULL)'.format(table,table),con=pyodbc.connect(connstring))
                else:  
                    df = pd.read_sql('select id ,issuenum , project from {}'.format(table),con=pyodbc.connect(connstring))
                df['source'] = server
                return df
            except:
                pd.DataFrame()
        if table =='bug':
            try:
                df = pd.read_sql('Select bugID as id, bugID as issuenum , ProjectID as project from {}'.format(table),con=pyodbc.connect(connstring))
                df['source'] = server
                return df
            except:
                pd.DataFrame()
    elif servertype == 'MySQL':
        try:
            con = pymysql.connect(host=connstring['host'],
                             user=connstring['user'],
                             password=connstring['password'],
                             db=connstring['db'],
                             cursorclass=pymysql.cursors.DictCursor)
            df = pd.read_sql('Select id, issuenum , project from {}'.format(table),con=con)
            df['source'] = server
            return df
        except:
            pd.DataFrame()
    elif servertype == 'PostgreSql':
        #try:
        con = psycopg2.connect(host=connstring['host'],
                             user=connstring['user'],
                             password=connstring['password'],
                             dbname=connstring['dbname']
                             )
        df = pd.read_sql('select id, issuenum , project from {}'.format(table),con=con)
        df['source'] = server
        return df
    #elif servertype == 'PostgreSqlPass':
        #try:
        #con = psycopg2.connect(host=connstring['host'],
                             #user=connstring['user'],
                             #dbname=connstring['dbname']
                             #)
        #df = pd.read_sql('select id, issuenum , project from {}'.format(table),con=con)
        #df['source'] = server
        #return df

#### Store the results

In [9]:
results = {}

In [11]:
#3min14sec
print('Picking Bugs from Sources')
for k,v in connections.items():
    #print (k,v)
    results[k]=query(k,v[0],v[2],v[1])

Picking Bugs from Sources


OperationalError: could not translate host name "jira-itoys-ro.c0hiis7ptxfm.us-west-2.rds.amazonaws.com" to address: Unknown host


### Check from which server the data was not imported

In [ ]:
for k,v in results.items():
    try:
        print(k, v.shape)
    except:
        print(k, 'ERROR')

In [ ]:
print('Bugs Imported Successfully from Sources')
final_table = pd.DataFrame()

In [ ]:
for i in results.values():
    final_table = final_table.append(i,ignore_index=True)

# Insert the data into a table and update the insert date

In [ ]:
engine = create_engine ('mssql+pyodbc://cds-hub-30.ad.ea.com/'+'DAPlayground'+'?driver=SQL+Server',legacy_schema_aliasing=False)

In [ ]:
check = pd.merge(final_table, existing, how='left', left_on=['source','project','id'], right_on=['source','project','id'])

# Missing issues

MISSINGGGG = check[check['issuenum_y'].isnull()]

cntMISSINGGGG = MISSINGGGG.groupby(['source','project']).count()

cntMISSINGGGG

cntMISSINGGGG.to_csv('cntMISSINGGGG.CSV')

is_12001 = MISSINGGGG['project'] == 12001

counterrors_13101 = MISSINGGGG[is_13101]

##export to csv if needed
counterrors_13101.to_csv('counterrors_13101.csv')

In [ ]:
print('Importing mismatches')
check[check['issuenum_y'].isnull()].to_sql(name='Missing_Bugs', con=engine, if_exists = 'append', index=False, chunksize = 200)

In [ ]:
print('Mismatches Imported Successfully')
connection = engine.connect()

In [ ]:
connection.execute("""update Missing_Bugs
set insert_date = GETUTCDATE()
where insert_date IS NULL""")

In [ ]:
connection.close()

In [ ]:
con = pyodbc.connect (
    r'DRIVER={SQL Server Native Client 11.0};'
    r'SERVER=cds-hub-30.ad.ea.com;'
    r'DATABASE=DAPlayground;'
    r'Trusted_Connection=yes;'
)

In [ ]:
cur = con.cursor()

In [ ]:
cur.execute("""select * into previous_date from (select distinct Insert_date, dense_rank() OVER (ORDER BY Insert_date DESC) AS Previous_date from Missing_Bugs) as previous_date""")
cur.execute("""select * into latest_bugs from (select distinct issuenum_x, source, project from Missing_Bugs where Insert_date = (select max(Insert_date) from Missing_Bugs)) as latest_bugs""")

In [ ]:
cur.execute("""update a
set IsPersistent = 'YES'
from Missing_Bugs a
inner join latest_bugs b
on a.issuenum_x = b.issuenum_x
and a.source = b.source
and a.project = b.project
inner join (select distinct issuenum_x, source, project from Missing_Bugs where Insert_date = (select distinct Insert_date from previous_date where Previous_date = 2)) as  c
on b.issuenum_x = c.issuenum_x
and b.project = c.project
and b.source = c.source
where a.Insert_date = (select max(Insert_date) from Missing_Bugs)""")

In [ ]:
cur.execute("""drop table previous_date""")
cur.execute("""drop table latest_bugs""")

In [ ]:
con.commit()
con.close()

In [ ]:
print('Finished')

# Test a connection if it works

test_connection = pyodbc.connect(
    r'DRIVER={SQL Server Native Client 11.0};'
    r'SERVER=DICE-SQLDEV.dice.ad.ea.com;'
    r'DATABASE=jira;'
    r'Trusted_Connection=yes;'
    )

test_set = pd.read_sql("""select top 10 * from jiraissue""", con = test_connection)

test_connection = pymysql.connect(host='jira-repldb.sphd.io',
                             user='wwqapmosvc',
                             password='1qaz2wsx!QAZ@WSX',
                             db = 'jiradb',
                             cursorclass=pymysql.cursors.DictCursor)

test_set = pd.read_sql("""select top 10 * from jiraissue""", con = test_connection)

In [ ]:
con = pymysql.connect(host= 'rca-jiradb01.rws.ad.ea.com',
                             user = 'dtreporting',
                             password = 'EhVz,Bub%V&b4=FetRtc1cx9vG',
                             db = 'odin_jiradb',
                             cursorclass = pymysql.cursors.DictCursor)

In [ ]:
con = pymysql.connect(host= 'ca1-testrailrpt.ad.ea.com',
                             user = 'cdshubs',
                             password = 'cdshubs',
                             db = 'testrail',
                             cursorclass = pymysql.cursors.DictCursor)

In [ ]:
test = pd.read_sql('Select id, issuenum , project from jiraissue',con=con)

In [ ]:
test

In [ ]:
con.close()